## Import bibliotek.

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree

## Ćwiczenie 1.
Wczytajamy zbiory dotyczące danych sprzedażowych "Produktu A".

In [2]:
tr = pd.read_parquet('dane/tr.parquet')
va = pd.read_parquet('dane/va.parquet')

#### 1.1. Rzuć okiem na zbiór. Odrzuć niepotrzebne kolumny.

**Zanim zaczniesz, przeczytaj proszę wnioski z poprzednich zajęć, które podsumowałem poniżej.**

---
Dla przypomnienia, wnioski z poprzednich zajęć i analizy zbioru uczącego:
* Najmocniej skorelowana ze zmienną celu: "srednia_sprzedaz_0_13d" - 0.783053
* Ze zmienną "srednia_sprzedaz_0_13d" wysoce skorelowane są zmienne: srednia_sprzedaz_0_2d, srednia_sprzedaz_0_6d
* Zmienne do sprawdzenia:
    * "srednia_cena_ropy_0_6d" - mocniej skorelowana ze zmienną celu niż srednia_cena_ropy_0_29d
    * "rok_dzialalnosci" - z każdym rokiem sprzedaż nieco wzrasta.
    * Pozostałe zmienne oparte na dacie: święta, weekendy, etc.
---

Skorzystaj z metody `tr.drop(columns=[], inplace=True)`

PS. Pamiętaj, by identyczą operację przeprowadzić dla obu zbiorów.

In [3]:
to_drop = ['srednia_sprzedaz_0_2d', 'data_sprzedazy', 'data_odciecia', 'cena_ropy', 'dzien_tygodnia']

In [4]:
tr.drop(columns=to_drop, inplace=True)
va.drop(columns=to_drop, inplace=True)

In [5]:
tr.head()

,id_drogerii,sprzedaz,miasto,wojewodztwo,typ_drogerii,srednia_sprzedaz_0_6d,srednia_sprzedaz_0_13d,srednia_cena_ropy_0_6d,srednia_cena_ropy_0_29d,weekend,swieto,miesiac,rok_dzialalnosci,sprzedaz_minus_1t
36,1,1,Warszawa,Mazowieckie,D,2.428571,2.214286,96.050000,94.531667,False,False,Luty,1,97.98
37,1,1,Warszawa,Mazowieckie,D,2.428571,2.142857,96.378571,94.682000,False,False,Luty,1,97.65
38,1,2,Warszawa,Mazowieckie,D,2.142857,2.357143,96.708571,94.826000,False,False,Luty,1,97.46
39,1,4,Warszawa,Mazowieckie,D,1.571429,2.000000,97.038571,94.975667,True,False,Luty,1,97.46
40,1,1,Warszawa,Mazowieckie,D,1.285714,1.785714,97.368571,95.120333,True,False,Luty,1,97.46


#### 1.2. Zbiór będzie zawierać wiele zmiennych. Rozdzielmy więc X i y na dwa zbiory.
Utwórz zmienne `X_tr`, `X_va`, `y_tr` i `y_va`.

In [6]:
X_tr = tr.drop(columns='sprzedaz').copy()
X_va = va.drop(columns='sprzedaz').copy()

y_tr = tr.sprzedaz.copy()
y_va = va.sprzedaz.copy()

In [7]:
X_tr.head()

,id_drogerii,miasto,wojewodztwo,typ_drogerii,srednia_sprzedaz_0_6d,srednia_sprzedaz_0_13d,srednia_cena_ropy_0_6d,srednia_cena_ropy_0_29d,weekend,swieto,miesiac,rok_dzialalnosci,sprzedaz_minus_1t
36,1,Warszawa,Mazowieckie,D,2.428571,2.214286,96.050000,94.531667,False,False,Luty,1,97.98
37,1,Warszawa,Mazowieckie,D,2.428571,2.142857,96.378571,94.682000,False,False,Luty,1,97.65
38,1,Warszawa,Mazowieckie,D,2.142857,2.357143,96.708571,94.826000,False,False,Luty,1,97.46
39,1,Warszawa,Mazowieckie,D,1.571429,2.000000,97.038571,94.975667,True,False,Luty,1,97.46
40,1,Warszawa,Mazowieckie,D,1.285714,1.785714,97.368571,95.120333,True,False,Luty,1,97.46


#### 1.3.Wykonaj konwersję zmiennych kategorycznych do typu binarnego.
* Skorzystaj z funkcji `pd.get_dummies(X_tr, columns=[tu_wpisz_zmienne_do_przekonwertowania])`.
* Wykonaj konwersję dla zmiennych: `['id_drogerii', 'miasto', 'wojewodztwo', 'typ_drogerii', 'dzien_tygodnia', 'weekend', 'swieto', 'miesiac']`.
* Wykonaj konwersję zarówno dla zbioru X_tr, jak i X_va.

In [8]:
X_tr = pd.get_dummies(X_tr, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)
X_va = pd.get_dummies(X_va, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)

In [9]:
X_tr.head()

,srednia_sprzedaz_0_6d,srednia_sprzedaz_0_13d,srednia_cena_ropy_0_6d,srednia_cena_ropy_0_29d,rok_dzialalnosci,sprzedaz_minus_1t,typ_drogerii_B,typ_drogerii_C,typ_drogerii_D,typ_drogerii_E,...,miesiac_Kwiecień,miesiac_Lipiec,miesiac_Listopad,miesiac_Luty,miesiac_Maj,miesiac_Marzec,miesiac_Październik,miesiac_Sierpień,miesiac_Styczeń,miesiac_Wrzesień
36,2.428571,2.214286,96.050000,94.531667,1,97.98,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
37,2.428571,2.142857,96.378571,94.682000,1,97.65,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
38,2.142857,2.357143,96.708571,94.826000,1,97.46,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
39,1.571429,2.000000,97.038571,94.975667,1,97.46,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
40,1.285714,1.785714,97.368571,95.120333,1,97.46,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


## Ćwiczenie 2.

#### 2.1. Zbuduj prostu "niedouczony" model drzewa decyzyjnego. Będzie on naszym punktem odniesienia dla kolejnych modeli.
* Ustaw parametry `max_dept=3` i `min_samples_leaf=0.1`.

In [10]:
model_1 = DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)
model_1.fit(X_tr, y_tr)

DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)

#### 2.2. Sprawdź jakość modelu na zbiorze `X_va`.

In [11]:
pred_tr = model_1.predict(X_tr)
pred_va = model_1.predict(X_va)

score_tr = mean_squared_error(y_tr, pred_tr, squared=False)
score_va = mean_squared_error(y_va, pred_va, squared=False)

print('TR: ', np.round(score_tr, 2))
print('TE: ', np.round(score_va, 2))

TR:  2.63
TE:  2.73


Najlepsze wyniki, jakie udało się osiągnąć z użyciem regresji liniowej:
* MSE TR: 2.294
* MSE VA: 2.298

Jest więc miejsce na poprawę.

## Ćwiczenie 3.
#### 3.1. Wykonaj próbkowanie z użyciem metody `df.sample`.
* Zbiór jest dosyć długi (prawie 50 tys. obserwacji) i dosić szeroki (kilkadziesiąt zmiennych). Na potrzeby poszukiwania optymalnych parametrów przeprowadzimy próbkowanie.

In [12]:
X_tr_sampled = X_tr.sample(5000, random_state=2001)
y_tr_sampled = y_tr.loc[X_tr_sampled.index]

#### 3.2. Zdefiniuj siatką parametrów i ustal odpowiednią CV.

In [13]:
parametry = {
    'criterion': ["mse"],
    'max_depth': [4, 5, 6, 7, 8],
    'min_samples_split': [0.01, 0.05, 0.1],
    'min_samples_leaf': [30, 40, 50],
    'random_state': [2001]
}

In [14]:
model_2 = DecisionTreeRegressor()

In [15]:
gs = GridSearchCV(model_2, parametry, n_jobs=2, scoring='neg_mean_squared_error', cv=4)
gs.fit(X_tr_sampled, y_tr_sampled)

GridSearchCV(cv=4, estimator=DecisionTreeRegressor(), n_jobs=2,
             param_grid={'criterion': ['mse'], 'max_depth': [4, 5, 6, 7, 8],
                         'min_samples_leaf': [30, 40, 50],
                         'min_samples_split': [0.01, 0.05, 0.1],
                         'random_state': [2001]},
             scoring='neg_mean_squared_error')

#### 3.3. Naucz nowy model i przeprowadź weryfikację jego jakości.

In [16]:
model_2 = DecisionTreeRegressor(**gs.best_params_)
model_2.fit(X_tr, y_tr)

DecisionTreeRegressor(max_depth=6, min_samples_leaf=40, min_samples_split=0.05,
                      random_state=2001)

In [17]:
pred_tr = model_2.predict(X_tr)
pred_va = model_2.predict(X_va)

score_tr = mean_squared_error(y_tr, pred_tr, squared=False)
score_va = mean_squared_error(y_va, pred_va, squared=False)

print('TR: ', np.round(score_tr, 2))
print('TE: ', np.round(score_va, 2))

TR:  2.28
TE:  2.32


#### 3.4. Sprawdź jakie zmienne okazaly się istotne.

In [18]:
pd.Series(model_2.feature_importances_, index=X_tr.columns).sort_values(ascending=False)[0:10]

srednia_sprzedaz_0_13d    0.868136
swieto_True               0.070074
srednia_sprzedaz_0_6d     0.031293
id_drogerii_44            0.015118
weekend_True              0.014524
id_drogerii_32            0.000818
id_drogerii_12            0.000020
sprzedaz_minus_1t         0.000010
miasto_Grudziądz          0.000007
miesiac_Maj               0.000000
dtype: float64

## Ćwiczenie 4.
#### 4.1. Wczytaj zbiór "wages_categorical.csv", który znajduje się w katalogu "dane". Zawiera on mnóstwo zmiennych kategorycznych - idealnie dla drzewa decyzyjnego! :-)

In [20]:
wages_cat = pd.read_csv('dane/wages_categorical.csv')

#### 4.2. Przeprowadź konwersję zmiennych kategorycznych do kodowania *dummy coding*.

In [21]:
wages_cat = pd.get_dummies(wages_cat)

#### 4.3. Podziel zbiór na 2 części w proporcjach 2:1. Rozdziel X od y.

In [22]:
wages_tr, wages_va = train_test_split(wages_cat, random_state=2001)

In [23]:
X_tr = wages_tr.drop(columns=['wage']).copy()
X_va = wages_va.drop(columns=['wage']).copy()

y_tr = wages_tr.wage.copy()
y_va = wages_va.wage.copy()

#### 4.4. Zamodeluj zmienną "wage". Zacznij od bazowego modelu z parametrami `max_depth=3, min_samples_leaf=0.1`.

In [24]:
model_3 = DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)
model_3.fit(X_tr, y_tr)

DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)

In [25]:
pred_tr = model_3.predict(X_tr)
pred_va = model_3.predict(X_va)

score_tr = mean_squared_error(y_tr, pred_tr, squared=False)
score_va = mean_squared_error(y_va, pred_va, squared=False)

print('TR: ', np.round(score_tr, 2))
print('TE: ', np.round(score_va, 2))

TR:  35.08
TE:  37.3


#### 4.5. Poszukaj sub-optymalnych parametrów modelu używając `GridSearchCV`.

In [26]:
model_4 = DecisionTreeRegressor()

In [27]:
gs = GridSearchCV(model_4, parametry, n_jobs=2, scoring='neg_mean_squared_error', cv=4)
gs.fit(X_tr, y_tr)

GridSearchCV(cv=4, estimator=DecisionTreeRegressor(), n_jobs=2,
             param_grid={'criterion': ['mse'], 'max_depth': [4, 5, 6, 7, 8],
                         'min_samples_leaf': [30, 40, 50],
                         'min_samples_split': [0.01, 0.05, 0.1],
                         'random_state': [2001]},
             scoring='neg_mean_squared_error')

#### 4.6. Naucz nowy model i przeprowadź weryfikację jego jakości.

In [28]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 6,
 'min_samples_leaf': 40,
 'min_samples_split': 0.01,
 'random_state': 2001}

In [29]:
model_3 = DecisionTreeRegressor(**gs.best_params_)
model_3.fit(X_tr, y_tr)

DecisionTreeRegressor(max_depth=6, min_samples_leaf=40, min_samples_split=0.01,
                      random_state=2001)

In [30]:
pred_tr = model_3.predict(X_tr)
pred_va = model_3.predict(X_va)

score_tr = mean_squared_error(y_tr, pred_tr, squared=False)
score_va = mean_squared_error(y_va, pred_va, squared=False)

print('TR: ', np.round(score_tr, 2))
print('TE: ', np.round(score_va, 2))

TR:  32.81
TE:  36.29


#### 4.7. Sprawdź jakie zmienne okazaly się istotne.

In [31]:
pd.Series(model_3.feature_importances_, index=X_tr.columns).sort_values(ascending=False)[0:15]

education_5. Advanced Degree    0.425441
health_ins_1. Yes               0.187477
education_4. College Grad       0.127374
age                             0.117985
maritl_2. Married               0.041219
maritl_1. Never Married         0.032277
education_3. Some College       0.032038
health_ins_2. No                0.021684
health_2. >=Very Good           0.006605
jobclass_1. Industrial          0.005341
education_2. HS Grad            0.002559
race_3. Asian                   0.000000
race_4. Other                   0.000000
education_1. < HS Grad          0.000000
race_2. Black                   0.000000
dtype: float64